In [13]:
%matplotlib inline
from helper import *
import plotter as pltr

In [14]:
# Hyper-parameters
window_size = 50
step_size = int(0.1 * window_size)
dataset = 'SMD'

In [15]:
if dataset == 'SMD':
    GROUP = 1
    ENTITY = 2
    SMD_BASE_PATH = 'Dataset/SMD'

    X_train = load_data('%s/train/machine-%d-%d.txt' % (SMD_BASE_PATH, GROUP, ENTITY), header=False)
    X_train.columns = ['m%d' % i for i in range(X_train.shape[1])]
    X_train.index = pd.date_range('2021/03/02', '2021/03/21', periods=X_train.shape[0])
    X_train.index.name = 'timestamp'

    X_test = load_data('%s/test/machine-%d-%d.txt' % (SMD_BASE_PATH, GROUP, ENTITY), header=False)
    X_test.columns = ['m%d' % i for i in range(X_test.shape[1])]
    X_test.index = pd.date_range('2021/03/21', '2021/4/8', periods=X_test.shape[0])
    X_test.index.name = 'timestamp'

    y_true = pd.read_csv('Dataset/SMD/test_label/machine-%d-%d.txt' % (GROUP, ENTITY), header=None)
    y_true.columns = ['label']
    y_true.index = X_test.index

In [16]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_true = y_true.to_numpy()
print("window size:", window_size)

window size: 50


In [17]:
# load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
from pymc3.distributions.timeseries import AR as ar
from scipy import optimize

sns.set_context('talk')

def create_model(X):
    with pm.Model() as model_pmc3:
        phi = pm.Beta("phi", 2., 2.)
        phi = 2*phi - 1
        sigma =  pm.InverseGamma('sigma', 2.5, 0.025)
        mu =  pm.Normal("mu", mu=0., sigma=1.)
        h = ar('h', rho=[mu*(1-phi), phi], sigma=sigma ** 2, shape=X.shape)
        y = pm.Normal('y', 0., pm.math.exp(h/2), observed=X)
        return model_pmc3

stochastic_vol_model = create_model(X_train)

In [18]:
# run NFVI
nf = "scale-loc"

with stochastic_vol_model:
    inference = pm.NFVI(nf, jitter=0.01)

inference.fit(3000, obj_optimizer=pm.adam(learning_rate=.1), obj_n_mc=20)

Interrupted at 315 [10%]: Average Loss = -5.9868e+05


In [ ]:
n_smpl = 1000
traceNF = inference.approx.sample(n_smpl)

In [ ]:
with stochastic_vol_model:
    posterior_predictive_nf = pm.sample_posterior_predictive(traceNF)